## TODO
 - [ ] Remover músicas instrumentais (elas não tem letra)
 - 

In [1]:
import numpy as np
import pandas as pd
#from songs_analyzer.config import get_root_path
#import nltk
#from nltk.corpus import stopwords
#from nltk.stem.snowball import PortugueseStemmer
'''
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
'''

'\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.model_selection import cross_validate\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.neural_network import MLPClassifier\nfrom sklearn.svm import SVC\nfrom sklearn.naive_bayes import MultinomialNB\n'

In [8]:
#pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 26.7 MB 35 kB/s             
     |████████████████████████████████| 306 kB 76.5 MB/s            
     |████████████████████████████████| 39.3 MB 39.9 MB/s            
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=ffca5e01781525fc4f3719845a6f382af39df7f4e5db33e13344255f7cf9b4c5
  Stored in directory: /home/ajsalmeida/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [ ]:
'''def join_lyrics(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ', '.join(x))
    return df

def remove_unknown_genders(df):
    unknown_genders = ['L', 'T','J', 'K','Z', 'M', 'R', 'B', 'V', 'E', 'G', 'F', 'S', 'D', 'H', 'P', 'C', 'A']
    return df.query("gender not in @unknown_genders")

def remove_small_gender(df):
    gender_count = df.value_counts('gender')
    values = gender_count[gender_count.values < 50].index
    return df.query("gender not in @values")

def transform_title(df):
    df['title'] = df['title'].apply(lambda x: x[0] if isinstance(x, list) else x)
    return df

def remove_instrumental_music(df):
    values = ['Instrumental', '(Instrumental)', '[Instrumental]', "", "instrumental"]
    return df.query("lyric not in @values")

def remove_small_letters(df):
    df = df.query('lyric.str.len() > 100')
    return df[df.lyric.apply(lambda x: len(set(x.split())) > 40)]
    '''
    

In [ ]:
'''
df = (
    pd
    .read_json(get_root_path() / "data/raw/songs.json")
    .pipe(join_lyrics, ['lyric'])
    .pipe(remove_unknown_genders)
    .pipe(remove_small_gender)
    .pipe(transform_title)
    .pipe(remove_instrumental_music)
    .pipe(remove_small_letters)
    
)
df.reset_index(drop=True, inplace=True)
df.drop_duplicates(subset='lyric', keep="first", inplace=True)
df.head()
'''

In [ ]:
#df.shape

In [ ]:
'''
X = df['lyric']
y = df['gender']
'''

Talvez seja bom capturar mais músicas dos gêneros que saíram pouco e são conhecidos, como `forró`, `Sertanejo`, etc.

In [ ]:
'''stop_words = stopwords.words('portuguese')
stemmer = PortugueseStemmer()
analyzer = TfidfVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

stem_vectorizer = TfidfVectorizer(
    analyzer=stemmed_words,
    stop_words=stop_words,
    max_features=5000
)
transformer = Pipeline(
    steps= [
        ('stemmer_tfidf', stem_vectorizer)
    ]
)
'''

In [ ]:
#X_vector = transformer.fit_transform(X)

In [ ]:
'''from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=10)

neigh.fit(X_vector)
'''

In [ ]:
#df.query('title == "')

In [ ]:
'''
letra = df.query('title == "De Quem É a Culpa?"').lyric.to_list()
songs = neigh.kneighbors(transformer.transform(letra), return_distance=False)
df.iloc[songs[0]]
'''

In [ ]:
#df.query('title == "Blood"')

In [ ]:
'''
decisiontree = DecisionTreeClassifier()

mlp = MLPClassifier(
    hidden_layer_sizes=(100,), 
    activation='relu',
    solver = 'adam',
    alpha=0.001
)

# clf = StackingClassifier(
#     estimators=[
#         ('decisiontree',decisiontree),
#         ('lr', lr),
#         ('mlp', mlp),
#         ('svc',svc)
#     ],
#     final_estimator=svc,
#     cv=10
# )

models = {
    'nb': MultinomialNB(),
    # 'lr': LogisticRegression(multi_class='multinomial', n_jobs=-1, max_iter=1000),
    'svc':SVC(probability=True, max_iter=1000),
    'decisiontree': DecisionTreeClassifier(),
    # 'mlp': mlp,
    # 'stacked': clf
}
'''

In [ ]:
'''
results = {}
for model_name, model in models.items():
    reg = Pipeline(
            steps=[
                ('preprocessor', transformer),
                ('regressor', model)
            ]
        )
    scores = cross_validate(reg, X, y, scoring=[ 'accuracy'], cv=5)
    results[model_name] = {
        # 'f1': scores['test_f1'],
        'accuracy': scores['test_accuracy']
    }
    print(results)
'''

In [15]:
df = pd.read_csv('../data/raw/music_genre.csv',usecols=[
    "artist_name",
    "track_name",
    "acousticness",
    "danceability",
    "energy",
    "instrumentalness",
    "key",
    "liveness",
    "speechiness",
    "music_genre"
])
df.sample(5)

,artist_name,track_name,acousticness,danceability,energy,instrumentalness,key,liveness,speechiness,music_genre
5505,"Fear, and Loathing in Las Vegas",The Answer for Unequal World,0.000338,0.320,0.964,0.001470,G#,0.415,0.0721,Anime
22883,Alan Jackson,What A Friend We Have In Jesus,0.903000,0.449,0.174,0.000000,D,0.124,0.0292,Country
49861,empty_field,XXX. FEAT. U2.,0.138000,0.568,0.619,0.000002,C#,0.110,0.4140,Hip-Hop
29720,Justin Timberlake,My Love (feat. T.I.),0.277000,0.772,0.680,0.000012,B,0.682,0.2240,Rap
49175,Metro Boomin,Borrowed Love (feat. Swae Lee & WizKid),0.440000,0.520,0.594,0.000000,B,0.185,0.4620,Hip-Hop


In [22]:
df.query('track_name == "What A Friend We Have In Jesus"')

,artist_name,track_name,acousticness,danceability,energy,instrumentalness,key,liveness,speechiness,music_genre
22883,Alan Jackson,What A Friend We Have In Jesus,0.903,0.449,0.174,0.0,D,0.124,0.0292,Country


In [19]:
X = df[['acousticness','danceability','energy','instrumentalness','key','liveness','speechiness']]

In [18]:
X

,acousticness,danceability,energy
0,0.00468,0.652,0.941
1,0.01270,0.622,0.890
2,0.00306,0.620,0.755
3,0.02540,0.774,0.700
4,0.00465,0.638,0.587
...,...,...,...
50000,0.03340,0.913,0.574
50001,0.15700,0.709,0.362
50002,0.00597,0.693,0.763
50003,0.08310,0.782,0.472
